In [16]:
import os
import sys
import csv
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from keras.regularizers import l2

In [26]:
df = pd.read_csv("../data/train_validation_cardio_tox_data.csv")
total_activity = df.loc[:,'ACTIVITY']
total_smiles = df.loc[:,'smiles']
X = pd.read_csv("../data/DESC_data/train_validation_cardio_tox_data_extract_features.csv")
x_label = list(X.columns)
y = total_activity
print("Total Row: ",df.shape[0])
print("Total Column: ", df.shape[1])
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

model = Sequential()
model.add(Dense(2000, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.0001), input_shape=(1826,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1000, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.0001), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=1, batch_size=10)
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))
model.summary()

checkpoint_path = "../saved_model/desc_model_train/ckpt_file/trained_cardio_tox_fingerprint_model.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
X = np.array(X)

model.fit(X, 
          y,  
          epochs=10,
          validation_data=(X, y),
          callbacks=[cp_callback])

model.save('../saved_model/desc_model_train/whole_model/cardio_tox_fingerprint_model')


/var/folders/x6/r97phsxs5qb8n7m2m39tbx0m0000gn/T/ipykernel_81658/4039657630.py:1: DtypeWarning: Columns (118,119,120,124,125,126,130,131,132,136,137,138,142,143,144,148,149,150,154,155,156,160,161,162,166,167,168,172,173,174,268,269,270,274,275,276,280,281,282,286,287,288,292,293,294,298,299,300,304,305,306,310,311,312,316,317,318,322,323,324,327,328,329,332,333,334,337,338,339,342,343,344,347,348,349,352,353,354,357,358,359,362,363,364,367,368,369,372,373,374,377,378,379,382,383,384,387,388,389,392,393,394,397,398,399,402,403,404,407,408,409,412,413,414,417,418,419,422,423,424,480,481,482,483,493,494,495,496,720) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/train_validation_cardio_tox_data.csv")


Total Row:  12620
Total Column:  997
355/355 [==============================] - 4s 10ms/step - loss: 65935.1797 - accuracy: 0.5378
Accuracy: 53.78
Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_60 (Dense)             (None, 2000)              3654000   
_________________________________________________________________
dropout_18 (Dropout)         (None, 2000)              0         
_________________________________________________________________
dense_61 (Dense)             (None, 1000)              2001000   
_________________________________________________________________
dropout_19 (Dropout)         (None, 1000)              0         
_________________________________________________________________
dense_62 (Dense)             (None, 200)               200200    
_________________________________________________________________
dense_63 (Dense)             (None, 1)

In [31]:
new_model = tf.keras.models.load_model('../saved_model/desc_model_train/whole_model/cardio_tox_fingerprint_model')
new_model.summary()
X_test_data = pd.read_csv('../data/DESC_data/external_test_set_pos_extract_features.csv')
acc = new_model.predict(X_test_data, verbose=2)
print(acc)

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_60 (Dense)             (None, 2000)              3654000   
_________________________________________________________________
dropout_18 (Dropout)         (None, 2000)              0         
_________________________________________________________________
dense_61 (Dense)             (None, 1000)              2001000   
_________________________________________________________________
dropout_19 (Dropout)         (None, 1000)              0         
_________________________________________________________________
dense_62 (Dense)             (None, 200)               200200    
_________________________________________________________________
dense_63 (Dense)             (None, 1)                 201       
Total params: 5,855,401
Trainable params: 5,855,401
Non-trainable params: 0
___________________________________________